In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"


In [3]:
import os
import gdown
from zipfile import ZipFile
from tqdm import tqdm

file_id = "1hi1dibkHyFbaxAteLlZJw6r3g9ddd4Lf"
url = f"https://drive.google.com/uc?id={file_id}"

download_path = "./dataset/dacon_dataset.zip"
extract_path = "./dataset/dacon_dataset"

if not os.path.exists(extract_path):
    os.makedirs(extract_path)

gdown.download(url, download_path, quiet=False)

with ZipFile(download_path, 'r') as zip_ref:
    for file in tqdm(zip_ref.namelist(), desc='Extracting files'):
        zip_ref.extract(file, extract_path)

os.remove(download_path)

print(f"File downloaded and extracted to {extract_path}")


Downloading...
From (original): https://drive.google.com/uc?id=1hi1dibkHyFbaxAteLlZJw6r3g9ddd4Lf
From (redirected): https://drive.google.com/uc?id=1hi1dibkHyFbaxAteLlZJw6r3g9ddd4Lf&confirm=t&uuid=19f496fc-41e8-4478-b6a1-2508f41c8bed
To: /home/hyj/ChanHyung/Audio/DACON_fake_voice_detection/dataset/dacon_dataset.zip
100%|██████████| 3.31G/3.31G [02:07<00:00, 26.0MB/s]
Extracting files: 100%|██████████| 106708/106708 [00:24<00:00, 4304.77it/s]


File downloaded and extracted to ./dataset/dacon_dataset


# Denosing

In [2]:
import glob
import math
import os
import tempfile
import time
from typing import List, Optional, Tuple, Union

import gradio as gr
import matplotlib.pyplot as plt
import numpy as np
import torch
from loguru import logger
from PIL import Image
from torch import Tensor

from df import config
from df.enhance import enhance, init_df, load_audio, save_audio
from df.io import resample

from torchaudio import AudioMetaData 

/home/hyj/anaconda3/envs/DACON_DEEPFAKE/lib/python3.11/site-packages/df/io.py:9: UserWarning: `torchaudio.backend.common.AudioMetaData` has been moved to `torchaudio.AudioMetaData`. Please update the import path.
  from torchaudio.backend.common import AudioMetaData


In [3]:
os.getcwd()

'/home/hyj/ChanHyung/Audio/DACON_fake_voice_detection'

In [4]:
import pandas as pd

base_path = os.getcwd()
train_df = pd.read_csv(f'{base_path}/dataset/dacon_dataset/train.csv')
test_df = pd.read_csv(f'{base_path}/dataset/dacon_dataset/test.csv')

In [5]:
import torch
from tqdm.auto import tqdm
import shutil

import warnings
warnings.filterwarnings(action='ignore')

dir_name = 'test_bandstop_denoising'
os.makedirs(f"./dataset/dacon_dataset/{dir_name}", exist_ok=True)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model, df, _ = init_df(os.path.join(base_path, 'DeepFilterNet2'), config_allow_defaults=True)
model = model.to(device=device).eval()
test_x = os.listdir(os.path.join(base_path, 'dataset', 'dacon_dataset', 'test'))

for i in tqdm(range(len(test_x))):
    id = test_x[i].split('/')[-1].split('.')[0]
    try: 
        path = os.path.join(base_path, 'dataset', 'dacon_dataset', 'test', test_x[i])
        sr = config("sr", 32000, int, section="df")
        sample, meta = load_audio(path, sr)
        enhanced = enhance(model, df, sample)

        lim = torch.linspace(0.0, 1.0, int(sr * 0.15)).unsqueeze(0)
        lim = torch.cat((lim, torch.ones(1, enhanced.shape[1] - lim.shape[1])), dim=1)
        enhanced = enhanced * lim
        if meta.sample_rate != sr:
            enhanced = resample(enhanced, sr, meta.sample_rate)
            sample = resample(sample, sr, meta.sample_rate)
            sr = meta.sample_rate

        noisy_wav = os.path.join(base_path, 'noisy_wav1557.wav')
        save_audio(noisy_wav, sample, sr)
        enhanced_wav = os.path.join(base_path, 'enhanced_wav1557.wav') 
        save_audio(enhanced_wav, enhanced, sr)
        noisy_wav = os.path.join(base_path, 'dataset', 'dacon_dataset', dir_name, f'noisy_{id}.wav')
        shutil.move(os.path.join(base_path, 'noisy_wav1557.wav'), noisy_wav)
        enhanced_wav = os.path.join(base_path, 'dataset', 'dacon_dataset', dir_name, f'enhanced_{id}.wav')
        shutil.move(os.path.join(base_path, 'enhanced_wav1557.wav') ,  enhanced_wav)
    except:
        print(f"너무 짧아서 안됨: {id}")

        

2024-07-23 02:41:09 | INFO     | DF | Running on torch 2.3.0
2024-07-23 02:41:09 | INFO     | DF | Running on host cvlab
2024-07-23 02:41:09 | INFO     | DF | Loading model settings of DeepFilterNet2
2024-07-23 02:41:09 | INFO     | DF | Initializing model `deepfilternet2`


fatal: not a git repository (or any of the parent directories): .git


2024-07-23 02:41:10 | INFO     | DF | Found checkpoint /home/hyj/ChanHyung/Audio/DACON_fake_voice_detection/DeepFilterNet2/checkpoints/model_96.ckpt.best with epoch 96
2024-07-23 02:41:10 | INFO     | DF | Running on device cuda:0
2024-07-23 02:41:10 | INFO     | DF | Model loaded


  0%|          | 0/50000 [00:00<?, ?it/s]

2024-07-23 02:41:10 | WARNING  | DF | Audio sampling rate does not match model sampling rate (32000, 48000). Resampling...


# Speech Segmentation

In [16]:
audio_file_list = [os.path.join(base_path, 'dataset', 'dacon_dataset', 'test_bandstop_denoising', i) \
                   for i in os.listdir(os.path.join(base_path, 'dataset', 'dacon_dataset', 'test_bandstop_denoising')) if "noisy" not in i]

In [19]:
import torchaudio
from speechbrain.inference.VAD import VAD
import os
import json
from tqdm import tqdm

# VAD 모델 초기화
vad_model = VAD.from_hparams(source="speechbrain/vad-crdnn-libriparty", savedir="pretrained_models/vad-crdnn-libriparty")

# 오디오 파일 경로 리스트
audio_file_list = [os.path.join(base_path, 'dataset', 'dacon_dataset', 'test_bandstop_denoising', i) \
                   for i in os.listdir(os.path.join(base_path, 'dataset', 'dacon_dataset', 'test_bandstop_denoising')) if "noisy" not in i]


# 타겟 샘플링 레이트
target_sample_rate = 16000

# 음성 구간 정보를 저장할 디렉토리
output_dir = os.path.join(base_path, 'vad_bandstop_boundaries_test')
os.makedirs(output_dir, exist_ok=True)

# 오디오 파일 처리
for input_file in tqdm(audio_file_list):
    # Step 1: 오디오 파일 로드 및 샘플링 레이트 변환
    signal, original_sample_rate = torchaudio.load(input_file)
    transform = torchaudio.transforms.Resample(orig_freq=original_sample_rate, new_freq=target_sample_rate)
    resampled_signal = transform(signal)
    
    # 오디오 길이 계산 (초 단위)
    audio_length = resampled_signal.size(1) / target_sample_rate
    
    # 임시 파일로 저장
    temp_output_file = os.path.join(base_path, 'temp_resampled_audio_file.wav')
    torchaudio.save(temp_output_file, resampled_signal, target_sample_rate)
    
    # Step 2: 변환된 파일로 VAD 수행
    boundaries = vad_model.get_speech_segments(temp_output_file)
    
    # Step 3: 각 음성 구간의 시작 시간과 종료 시간을 기록
    base_filename = os.path.splitext(os.path.basename(input_file))[0]
    boundaries_list = []
    for boundary in boundaries:
        start, end = boundary[0].item(), boundary[1].item()
        boundaries_list.append({
            "start": start,
            "end": end
        })
    
    # JSON 파일로 저장
    output_data = {
        "audio_length": audio_length,
        "boundaries": boundaries_list
    }
    boundaries_filename = f"{base_filename}_boundaries.json"
    boundaries_filepath = os.path.join(output_dir, boundaries_filename)
    with open(boundaries_filepath, 'w') as f:
        json.dump(output_data, f)
    
    # print(f"Saved boundaries and audio length for {input_file} to {boundaries_filepath}")
    
    # 임시 파일 삭제
    os.remove(temp_output_file)

print("All boundaries and audio lengths have been saved.")


torchvision is not available - cannot save figures


100%|██████████| 50000/50000 [5:41:57<00:00,  2.44it/s]  

All boundaries and audio lengths have been saved.
